In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report

W1106 20:03:12.302000 61783 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
ds = load_dataset("lmarena-ai/arena-human-preference-140k")

In [3]:
train_ds = ds['train']
print(train_ds)

Dataset({
    features: ['id', 'model_a', 'model_b', 'winner', 'evaluation_session_id', 'evaluation_order', 'conversation_a', 'conversation_b', 'full_conversation', 'conv_metadata', 'category_tag', 'language', 'is_code', 'timestamp'],
    num_rows: 135634
})


In [14]:
def reorganize(dataset):
    df = dataset.to_pandas()
    
    # Create model_a rows
    df_a = df.rename(columns={
        'model_a': 'model_name',
        'conversation_a': 'conversation',
        'model_b': 'other_model'
    })[['model_name', 'conversation', 'other_model', 'winner', 'evaluation_session_id', 
        'evaluation_order', 'full_conversation', 'conv_metadata', 'category_tag', 
        'language', 'is_code', 'timestamp', 'id']]
    # Create model_b rows
    df_b = df.rename(columns={
        'model_b': 'model_name',
        'conversation_b': 'conversation',
        'model_a': 'other_model'
    })[['model_name', 'conversation', 'other_model', 'winner', 'evaluation_session_id', 
        'evaluation_order', 'full_conversation', 'conv_metadata', 'category_tag', 
        'language', 'is_code', 'timestamp', 'id']]
    # Combine and convert back
    df_combined = pd.concat([df_a, df_b], ignore_index=True)
    return df_combined

reorganized = reorganize(train_ds)
print(reorganized.head())

                              model_name  \
0                         gemini-2.5-pro   
1              claude-3-5-haiku-20241022   
2                                o3-mini   
3  claude-sonnet-4-20250514-thinking-32k   
4             claude-3-5-sonnet-20241022   

                                        conversation  \
0  [{'role': 'user', 'content': [{'type': 'text',...   
1  [{'role': 'user', 'content': [{'type': 'text',...   
2  [{'role': 'user', 'content': [{'type': 'text',...   
3  [{'role': 'user', 'content': [{'type': 'text',...   
4  [{'role': 'user', 'content': [{'type': 'text',...   

                               other_model    winner  \
0  claude-3-7-sonnet-20250219-thinking-32k   model_a   
1               claude-3-5-sonnet-20241022       tie   
2                          gemma-3n-e4b-it  both_bad   
3                           gemini-2.5-pro   model_a   
4                      mistral-medium-2505   model_b   

                  evaluation_session_id  evaluation_order  \


In [15]:
num_unique_models = len(set(reorganized['model_name']))
print(num_unique_models)

53


In [16]:
unique_models = sorted(set(reorganized['model_name']))
model_to_label = {model: idx for idx, model in enumerate(unique_models)}
label_to_model = {idx: model for model, idx in model_to_label.items()}
reorganized['model_label'] = reorganized['model_name'].map(model_to_label)

print(f"Model to label mapping: {model_to_label}")
print(reorganized.head())

Model to label mapping: {'amazon-nova-experimental-chat-05-14': 0, 'amazon.nova-pro-v1:0': 1, 'chatgpt-4o-latest-20250326': 2, 'claude-3-5-haiku-20241022': 3, 'claude-3-5-sonnet-20241022': 4, 'claude-3-7-sonnet-20250219': 5, 'claude-3-7-sonnet-20250219-thinking-32k': 6, 'claude-opus-4-20250514': 7, 'claude-opus-4-20250514-thinking-16k': 8, 'claude-sonnet-4-20250514': 9, 'claude-sonnet-4-20250514-thinking-32k': 10, 'command-a-03-2025': 11, 'deepseek-r1-0528': 12, 'deepseek-v3-0324': 13, 'gemini-2.0-flash-001': 14, 'gemini-2.0-flash-thinking-exp-01-21': 15, 'gemini-2.5-flash': 16, 'gemini-2.5-flash-lite-preview-06-17-thinking': 17, 'gemini-2.5-flash-preview-04-17': 18, 'gemini-2.5-pro': 19, 'gemini-2.5-pro-preview-03-25': 20, 'gemini-2.5-pro-preview-05-06': 21, 'gemma-3-27b-it': 22, 'gemma-3n-e4b-it': 23, 'gpt-4.1-2025-04-14': 24, 'gpt-4.1-mini-2025-04-14': 25, 'gpt-4o-2024-11-20': 26, 'gpt-4o-mini-2024-07-18': 27, 'grok-3-mini-beta': 28, 'grok-3-mini-high': 29, 'grok-3-preview-02-24': 3

In [ ]:
def clean_conversation(conversation_list):
    """
    Convert conversation list format to a single text string
    """
    
    text_parts = []
    for turn in conversation_list:
        if isinstance(turn, dict) and 'content' in turn and 'role' in turn:
            role = turn['role']
            # Extract text from content list
            for content_item in turn['content']:
                if isinstance(content_item, dict) and content_item.get('text'):
                    text = content_item['text']
                    text_parts.append(f"{role}: {text}")
    return " ".join(text_parts) if text_parts else None

In [26]:
print(type(reorganized['conversation'][0]))
print(clean_conversation(reorganized['conversation'][0]))

<class 'numpy.ndarray'>
['user: Compare Tormenta20 with DnD5e', 'assistant: Excellent question! Comparing **Tormenta20 (T20)** and **Dungeons & Dragons 5th Edition (D&D 5e)** is a fantastic way to understand different approaches to the d20 system. While they share a common ancestor (D&D 3rd Edition), they have evolved in vastly different directions.\n\nHere is a detailed comparison, broken down by key aspects of the games.\n\n---\n\n### **High-Level Philosophy**\n\n*   **D&D 5e:** Aims for **accessibility, streamlined play, and heroic fantasy**. It simplifies many of the complex rules of older editions to be welcoming to new players. Its design philosophy is "rulings, not rules," encouraging Dungeon Masters to make calls on the fly. It is built to be setting-agnostic, though the Forgotten Realms is its default.\n*   **Tormenta20:** Aims for **deep character customization, high-power fantasy, and an "anime/JRPG" feel**. It offers players a vast menu of options to build a unique characte

In [18]:
reorganized['conversation_text'] = reorganized['conversation'].apply(clean_conversation)
print(reorganized['conversation_text'])

0         None
1         None
2         None
3         None
4         None
          ... 
271263    None
271264    None
271265    None
271266    None
271267    None
Name: conversation_text, Length: 271268, dtype: object


In [11]:
reorganized = reorganized[reorganized['conversation_text'].notna()]
reorganized = reorganized[reorganized['conversation_text'].str.len() > 10]

print(f"Rows after cleaning: {len(reorganized)}")
print(f"\nSample cleaned conversation:\n{reorganized['conversation_text'].iloc[0][:500]}")


Rows after cleaning: 0


IndexError: single positional indexer is out-of-bounds

In [7]:
dataset = Dataset.from_pandas(reorganized[['conversation', 'model_label']])

{'conversation': [{'content': [{'image': None, 'mimeType': None, 'text': 'Compare Tormenta20 with DnD5e', 'type': 'text'}], 'num_tokens': None, 'role': 'user'}, {'content': [{'image': None, 'mimeType': None, 'text': 'Excellent question! Comparing **Tormenta20 (T20)** and **Dungeons & Dragons 5th Edition (D&D 5e)** is a fantastic way to understand different approaches to the d20 system. While they share a common ancestor (D&D 3rd Edition), they have evolved in vastly different directions.\n\nHere is a detailed comparison, broken down by key aspects of the games.\n\n---\n\n### **High-Level Philosophy**\n\n*   **D&D 5e:** Aims for **accessibility, streamlined play, and heroic fantasy**. It simplifies many of the complex rules of older editions to be welcoming to new players. Its design philosophy is "rulings, not rules," encouraging Dungeon Masters to make calls on the fly. It is built to be setting-agnostic, though the Forgotten Realms is its default.\n*   **Tormenta20:** Aims for **deep